In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [8]:
#!pip3 install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [51]:

from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException,ElementNotInteractableException,ElementClickInterceptedException

In [65]:
from selenium import webdriver
import time
import requests
import shutil
import os
from webdriver_manager.chrome import ChromeDriverManager

#Install driver
opts=webdriver.ChromeOptions()
#opts.headless=True
driver = webdriver.Chrome(ChromeDriverManager().install(),options=opts)
user = os.getlogin()

urls = []
Review=[]
Ratings=[]
urls = []
driver.get('https://www.amazon.in/')
#extracting links for different products having unique ID
tech_list=["professional camera" ,"laptop","headphone","smartwatch"]
for i in tech_list:
    search_bar = driver.find_element_by_id("twotabsearchtextbox")    # Locating searc_bar by id
    search_bar.clear()                                               # clearing search_bar
    search_bar.send_keys(i)                                   # sending user input to search bar
    search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')       # Locating search_button by xpath
    search_button.click()
    
    start_page = 0
    end_page = 10
    
    for page in range(start_page,end_page+1):
        try:
            page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
            #print(page_urls)
        # appending all the urls on current page to urls list
            for url in page_urls:
                url = url.get_attribute('href')     # Scraping the url from webelemen
                #print(url)
                urls.append(url)              # Appending the url to urls list
                #print(urls)
            #print("Product urls of page",str(page+1)," has been scraped for.",i)
    
    
            
        # Moving to next page
            nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
            if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
                nxt_button.click()                                                    # Clicking the next button
                time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
            elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
                print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
                break
            
        except StaleElementReferenceException as e:             # Handling StaleElement Exception   
            print("Stale Exception")
            next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
            driver.get(next_page)                               # ReLoading the next page
        

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [/Users/icg/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [66]:
urls

['https://www.amazon.in/Professional-Digital-Camcorder-Definition-Rotation/dp/B091P37YT8/ref=sr_1_1?dchild=1&keywords=professional+camera&qid=1624510874&sr=8-1',
 'https://www.amazon.in/Todays-Deal-Squaircle-2IN1-Camera/dp/B08TC39N2V/ref=sr_1_2?dchild=1&keywords=professional+camera&qid=1624510874&sr=8-2',
 'https://www.amazon.in/Professional-Colorful-Colour-Changing-Tripod/dp/B097KL18Z1/ref=sr_1_3?dchild=1&keywords=professional+camera&qid=1624510874&sr=8-3',
 'https://www.amazon.in/Canon-1500D-Digital-Camera-S18-55/dp/B07BS4TJ43/ref=sxin_9?asc_contentid=amzn1.osa.ee645fcc-008b-43ea-8bdd-a6fa75b479cb.A21TJRUUN4KGV.en_IN&asc_contenttype=article&ascsubtag=amzn1.osa.ee645fcc-008b-43ea-8bdd-a6fa75b479cb.A21TJRUUN4KGV.en_IN&creativeASIN=B07BS4TJ43&cv_ct_cx=professional+camera&cv_ct_id=amzn1.osa.ee645fcc-008b-43ea-8bdd-a6fa75b479cb.A21TJRUUN4KGV.en_IN&cv_ct_pg=search&cv_ct_we=asin&cv_ct_wn=osp-single-source-earns-comm&dchild=1&keywords=professional+camera&linkCode=oas&pd_rd_i=B07BS4TJ43&pd_rd

In [62]:
#Install driver
opts=webdriver.ChromeOptions()
#opts.headless=True
driver = webdriver.Chrome(ChromeDriverManager().install(),options=opts)
for j in urls:
    driver.get(j)  
    # Loading the webpage by url
    print("Scraping URL = ", j)
    #time.sleep(2)
    try:
        # click of "See all reviews" link to extract all reviews and ratings            
        review_link=driver.find_element_by_xpath("//div[@class='a-row a-spacing-medium']/a")
        #print(review_link)
        if review_link.text=="See all reviews":
            review_link.click()
        else:
            continue
    except ElementClickInterceptedException as s:
        print(s)
        
    try:    
        #scraping ratings//*[@id="customer_review-R1VU9KM799DRVW"]/div[2]/a[1]/i/span
        ratings=driver.find_elements_by_xpath("//div[@class='a-section celwidget']/div[2]/a[1]")
        #print(ratings)
        for b in ratings:
            if b =='':
                continue
            else:
                star=b.get_attribute('title')
                        #print(star)
                star_ext=star.split(' ')
                rating=int(float(star_ext[0]))
                print(rating)
                Ratings.append(rating)
                            
    except NoSuchElementException as e:
        print(e)
            
    try: #scraping reviews
                    
        reviews=driver.find_elements_by_xpath("//div[@class='a-section celwidget']/div[2]/a[2]/span")
        #print(reviews)
        for r in reviews:
            if r =='':
                continue
            else:
                Review.append(r.text)
                print(r.text)
    except NoSuchElementException as e:
        print(e)
                        

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [/Users/icg/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A03004261PZSSAJ9KRAK0&url=%2FTECNO-Spark-Storage-Battery-Camera%2Fdp%2FB096LS7N6Z%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dmobile%26qid%3D1624473766%26sr%3D8-1-spons%26psc%3D1&qualifier=1624473766&id=1608521012761527&widgetName=sp_atf
5
5
5
5
5
5
1
4
1
5
Value of Money 💖
Ram and finishing
No music player..lots of app
Best mobile with in this segment, when compared with other compitators..
Superb product in prise
Best mobile under 10k
Prosese is not good caimera 48 mp bool ke 5 mp se bhe bekaar hi
Better not best
बस ठीक है बढिया नहीं है
Best cemara
Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A09144133CAA4GKRKRBCL&url=%2FTecno-Spark-Alps-Blue-Storage%2Fdp%2FB095GWK44S%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3Dmobile%26qid%3D1624473766%26sr%3D8-2-spons%26psc%3D1&qualifier=1624473766&id=160852101276152

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='a-row a-spacing-medium']/a"}
  (Session info: chrome=91.0.4472.114)


In [63]:
print(len(Review),len(Ratings))

501 499


In [ ]:
df_amazon=pd.DataFrame({ 'Reviews':Review,
                         'Ratings':Ratings
                         })
    
print(df_amazon)

In [6]:
df_amazon["Ratings"].value_counts()

5    185
1     98
4     84
2     35
3     34
Name: Ratings, dtype: int64

In [9]:
df_amazon["Reviews"].unique()

array(['Value of Money 💖', 'No music player..lots of app',
       'Ram and finishing', 'Superb product in prise',
       'Best mobile under 10k',
       'Prosese is not good caimera 48 mp bool ke 5 mp se bhe bekaar hi',
       'Better not best', 'Amazing', 'Good for battery backup',
       'Best Value for Money in this Range. Go for it without any doubt.',
       'Samsung m31 all rounder best mobile.', 'Poor camera',
       'False ads, poor camera quality, inefficient battery management',
       'Serious Issue', 'Inferior quality and built', 'Not good at alll',
       "Samsung it's not fair .", 'Worst phone', 'Bad Product by Samsung',
       'earphones levu. and Mobile software working slow. So Mobile avarage. Amazon',
       'Oppo a31', 'Awesome', 'Osm',
       "Waste of money. Please don't buy it", 'Worthy n best product',
       'Complaint', 'No headphone in the box', 'Average!',
       'Very Good product', "Don't buy this phone",
       'One of the good choice for low budget phone'

In [3]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/icg/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [57]:
from selenium import webdriver
import time
import requests
import shutil
import os
from webdriver_manager.chrome import ChromeDriverManager

#Install driver
opts=webdriver.ChromeOptions()
#opts.headless=True

driver = webdriver.Chrome(ChromeDriverManager().install(),options=opts)
Reviews=[]
Ratings=[]
tech_list=["smartphone","laptop","smartwatch","camera","home theatre","printer"]

for i in tech_list:
    for page in range(1,15,1):
        web= "https://www.flipkart.com/search?q=" +str(i)+ "&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=" + str(page)
        print(web)
        driver.get(web)
        try:
    #locating ratings
            urls=driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
            url=[]
            for u in urls:
                url.append(u.get_attribute('href'))
            for j in url:
                driver.get(j)  
                try: #scraping reviews
                    review_link=driver.find_element_by_xpath("//div[@class='col JOpGWq']/a")
                    review_link.click()
                    time.sleep(2)
                    reviews=driver.find_elements_by_xpath("//div[@class='col _2wzgFH K0kLPL']/div[1]/p")
                    #print(reviews)
                    for r in reviews:
                        if r =='':
                            continue
                        else:
                            Reviews.append(r.text)
                            print(r.text)
                except NoSuchElementException as e:
                    print(e)
            
                try: #scraping ratings
                    ratings=driver.find_elements_by_xpath("//div[@class='col _2wzgFH K0kLPL']/div[1]/div")
                    #print(ratings)
                    for t in ratings:
                        if t =='':
                            continue
                        else:
                            Ratings.append(t.text)
                            print(t.text)
    
                except NoSuchElementException as e:
                    print(e)
            
        
            
        except NoSuchElementException as s:
            print(s)

    
print(len(Reviews),len(Ratings))    





====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/icg/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


https://www.flipkart.com/search?q=smartphone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page=1
Just wow!
Worth the money
Perfect product!
Excellent
Brilliant
Terrific
Must buy!
Good quality product
Pretty good
Nice
5
4
5
5
5
5
5
4
4
3
Good quality product
Really Nice
Perfect product!
Terrific purchase
Nice product
Worth every penny
Highly recommended
Does the job
Classy product
Mind-blowing purchase
4
4
5
5
4
5
5
3
5
5
Good quality product
Really Nice
Perfect product!
Terrific purchase
Nice product
Worth every penny
Highly recommended
Does the job
Classy product
Mind-blowing purchase
4
4
5
5
4
5
5
3
5
5
Wonderful
Wonderful
Simply awesome
Delightful
Awesome
Does the job
Very Good
Terrific
Just wow!
Really Nice
4
5
5
4
5
3
4
5
5
4
Brilliant
Brilliant
Good
Pretty good
Wonderful
Classy product
Nice
Simply awesome
Excellent
Nice product
5
5
3
4
4
5
3
5
5
4
Good quality product
Really Nice
Perfect product!
Terrific purchase
Nice product
Worth every penny
Highly r

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=91.0.4472.106)


In [17]:
#creating a dataframe
df_flipkart=pd.DataFrame({
"Reviews":Reviews,
"Ratings":Ratings
})
#printing dataframe
df_flipkart 

,Reviews,Ratings
0,Just wow!,5
1,Worth the money,4
2,Perfect product!,5
3,Excellent,5
4,Brilliant,5
...,...,...
6922,Classy product,5
6923,Worth every penny,5
6924,Fabulous!,5
6925,Great product,5


In [72]:
df_flipkart["Ratings"].value_counts()

5    4460
4    1411
1     463
3     403
2     190
Name: Ratings, dtype: int64

In [73]:
df_flipkart.to_csv("Ratings.csv",header=True)